In [ ]:
# import packages and functions needed for using the notebook

import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))
from IPython.display import HTML, Image, Javascript, display, Markdown, Math, Latex

import ipywidgets as widgets

import xml.etree.ElementTree as ET

import warnings
warnings.filterwarnings("ignore")

from traitlets import traitlets

# Style for widgets labels (solves problem with too short label)
style = {'description_width': 'initial'}
# Layout, so Button labels are not too long
layout = {"width" : "max-content"}
# Layout for boxes that contain several widgets
box_layout = {"flex-wrap" : "initial",  "flex_flow" : "column", "align_content" : "space-between", "border" : "solid", "width" : "initial"}

In [ ]:
# import packages for ANN use

# basic packages
import numpy as np
import tensorflow as tf
import random as rn
import datetime
import pandas as pd

# specific subpackages of keras
from keras.layers import Input, Dense
from keras.models import Model
from keras import optimizers
from keras import callbacks

# auxiliary functions
from assets.py.tesa_uebung_ann_auxiliary import load_data, get_input_ratio, plot_training_history, \
                                      mean_absolute_percentage_error, postprocess_prediction_results, \
                                      plot_activations, show_data_excerpt, plot_data_excerpt

# All  necessary dependencies:
# For ANN: numpy, random, xlrd, matplotlib, pandas, scikit-learn, tensorflow 1.X

In [ ]:
# initialize randomness for reproducible results
np.random.seed(42)
rn.seed(12345)
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1,
                              inter_op_parallelism_threads=1)

tf.random.set_seed(2)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

In [ ]:
# This cell contains functions for running certain cells

def run_next(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))
    
def run_next_two(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+3)'))

def run_next_three(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+4)'))
     
def run_before(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()-3, IPython.notebook.get_selected_index())-1'))

def run_all_below(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'))
    
def run_current_and_all_below(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index(), IPython.notebook.ncells())'))
    
def run_all(ev):
    display(Javascript('IPython.notebook.execute_cell_range(0, IPython.notebook.ncells())'))
    
def run_all_alternative():
    display(Javascript('IPython.notebook.execute_cell_range(0, IPython.notebook.ncells())'))

In [ ]:
%%html

<!-- This cell contains all HTML output for displaying the content correctly -->

<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
        }
    );
</script>
<style>
    div.output_area .MathJax_Display {
        text-align: center !important;
    }
    .container { 
        #width:100% !important; 
    }
</style>

In [ ]:
# This cell creates a button to toggle the GUI code.

toggleGUI = HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" 
value="Klicken Sie hier, um den Code des GUI-Java Scripts anzuzeigen/auszublenden!"></form>''')
display(toggleGUI)

In [ ]:
# This cell creates a dropdown menu for selecting the language
language_dropdown = widgets.Dropdown(
    options=['Deutsch', 'English'],
    value='Deutsch',
    description='Select your language:',
    style=style
)

# Create Button widget to proceed
button_confirm_language = widgets.Button(description="OK", button_style='success')

# Display all of the widgets 
display(widgets.HBox(children = [language_dropdown, button_confirm_language]))


button_confirm_language.on_click(run_all_below)

In [ ]:
# This cell sets the selected the language .xml-file corresponding to the language chosen and parses it

if language_dropdown.value == 'Deutsch':
    filepath_language_xml = 'assets/strings/ann_deutsch.xml'
elif language_dropdown.value == 'English':
    filepath_language_xml = 'assets/strings/ann_english.xml'


tree = ET.parse(filepath_language_xml)
language_strings = tree.getroot()

# Function to get a string by ID from the parsed xml file
def string(name):
    try:
        value = language_strings.find("string/[@name='"+name+"']").text
        return value
    except AttributeError as error:
        print('##### No string with this name in xml file #####')

In [ ]:
# This cell creates a button for resetting the notebook

buttonReset = widgets.Button(description=string('txt_Einstellungen_Reset'),
                            button_style='success')


buttonReset.on_click(run_all)
display(buttonReset)

In [ ]:
# This cell displays heading and logo

display(Markdown(string('hd_Einführung')))

<img src="assets/img/rwth_eerc_rgb.png" alt="JERI-Logo" style="display: block;  margin-left: auto;  margin-right: auto;  width: 80%;"/>

In [ ]:
display(Markdown(string("hd_Einleitung")))
display(Markdown(string("txt_Einleitung")))
display(Markdown(string("txt_Einleitung2")))

In [ ]:
display(Markdown(string("hd_Daten")))
display(Markdown(string("txt_Daten1")))
display(Markdown(string("txt_Daten2")))

In [ ]:
# Create  widget for selecting dates to show
start_date_picker = widgets.DatePicker(description =string("lbl_Startdatum"), value = datetime.date(2012,1,1), style = style)
end_date_picker = widgets.DatePicker(description =string("lbl_Enddatum"), value = datetime.date(2016,12,31), style = style)

# Create Button widgets to proceed
button_confirm_date = widgets.Button(description=string('lbl_ButtonOK'), button_style='success')

# Display all of the widgets 
display(widgets.HBox(children = [start_date_picker, end_date_picker, button_confirm_date]))

button_confirm_date.on_click(run_next)

In [ ]:
# This cell displays the graphs of the data

# Set start and end date
start = start_date_picker.value
end = end_date_picker.value

# Names of columns in the dataset and helper dictionary to set titles in selected language
columns = ["windspeed", "temperature", "radiation_direct", "radiation_diffuse", "load"]
titles_data = {"English" : {"windspeed" : "Wind Speed", "temperature" : "Temperature", "radiation_direct" : "Direct Radiaion", "radiation_diffuse" : "Diffuse Radiation", "load" : "Electric load" }, 
               "Deutsch": { "windspeed" :  "Windgeschwindigkeit", "temperature" : "Temperatur", "radiation_direct" : "Direkte Sonneneinstrahlung", "radiation_diffuse" : "Diffuse Sonneneinstrahlung", "load" : "Elektrische Last"  }}

# Create widgets: output widgets to capture the plots, tab widget to house the plots
outs_data = [widgets.Output() for column in columns]
tab_data = widgets.Tab(children = outs_data)

# Fill widgets with data and display the plots
for i in range(len(columns)):
    tab_data.set_title(i, titles_data[language_dropdown.value][columns[i]])
    with outs_data[i]:
        plot_data_excerpt("assets/data/tesa_uebung_ann_data.csv", start_date = start, end_date = end, column = columns[i])

display(tab_data)

In [ ]:
display(Markdown(string("Abb_Daten")))

In [ ]:
display(Markdown(string("hd_Neuronales_Netz")))
display(Markdown(string("txt_Neuronales_Netz")))
display(Markdown(string("txt_Anpassungen_list")))

In [ ]:
display(Markdown(string('hd_Vorhersagehorizont')))
display(Markdown(string('txt_Vorhersagehorizont')))

In [ ]:
# Input text for specifying the output horizon
Input_output_horizon = widgets.BoundedIntText(value=168, min=24.0, max=336.0, step=1.0, description=string('lbl_InputHorizont'), style=style)

# Create Button widget to proceed
button_confirm_horizon = widgets.Button(description=string('lbl_ButtonOK'), button_style='success')

# Display the widgets in a Box widget
display(widgets.HBox(children = [Input_output_horizon, button_confirm_horizon]))

# Once the button is clicked, the next cells are run
button_confirm_horizon.on_click(run_next_two)

In [ ]:
output_horizon = Input_output_horizon.value

In [ ]:
display(Markdown(string('hd_Hidden_Layer')))
display(Markdown(string('txt_Hidden_Layer')))

<img src="assets/img/ANN_Architecture.png" alt="JERI-Logo" style="display: block;  margin-left: auto;  margin-right: auto;  width: 80%;"/>

In [ ]:
display(Markdown(string("Abb_Architektur")))

In [ ]:
display(Markdown(string('hd_Anzahl_Neuronen')))
display(Markdown(string('txt_Anzahl_Neuronen')))
display(Markdown(string('txt_Anzahl_Neuronen2')))

In [ ]:
# Input text for specifying the number of units in the hidden layers
Input_no_of_units = widgets.BoundedIntText(value=128, min=1, max=512, step=1, description=string('lbl_InputAnzahlUnits'), style=style)

# Create Button widget to proceed
button_confirm_units = widgets.Button(description=string('lbl_ButtonOK'), button_style='success')

# Display all of the widgets next to each other in a box
display(widgets.HBox(children = [Input_no_of_units, button_confirm_units]))

# Once the button is clicked, run next cell
button_confirm_units.on_click(run_next)

In [ ]:
no_of_units = Input_no_of_units.value
Input_no_of_units.observe(run_next, names='value')

In [ ]:
display(Markdown(string('hd_Anzahl_Hidden_Layer')))
display(Markdown(string('txt_Anzahl_Hidden_Layer')))

In [ ]:
display(Markdown(string('txt_Aktivierungen')))
plot_activations(["tanh", "sigmoid", "linear", "relu"], -10, 10)

In [ ]:
display(Markdown(string("Abb_Aktivierungen")))

In [ ]:
display(Markdown(string('txt_Aktivierungen2')))

In [ ]:
# Add Hidden Layers

# Create Slider Widget 
AddHiddenLayers = widgets.IntSlider(value=2, min = 1, max = 5, description=string('lbl_LayerHinzufügen'), continuous_update=True, style=style)

# Create Button widget to confirm changes
button_confirm_layers = widgets.Button(description=string('lbl_ButtonOK'), button_style='success')

# Display all of the widgets next to each other in a box
display(widgets.HBox(children = [AddHiddenLayers, button_confirm_layers]))

button_confirm_layers.on_click(run_next_two)

In [ ]:
# Set no. of Hidden Layers
no_of_hidden_layers = AddHiddenLayers.value

# Create Tab-Widget and fill in with one activation dropdown widget per Hidden Layer
tab_activations = widgets.Tab()
tab_activations.children = [widgets.ToggleButtons(options=["tanh", "sigmoid", "relu", "linear"], value="tanh", \
                             description=string("lbl_InputActivierung"), style=style) for layer in range(no_of_hidden_layers+1)]

# Set titles for the tabs
for i in range(no_of_hidden_layers):
    tab_activations.set_title(i, "Hidden Layer {}".format(i+1))
tab_activations.set_title(no_of_hidden_layers, "Output layer")

# Create button widget for proceeding
button_confirm_activations = widgets.Button(description=string("lbl_ButtonOK"),
                            button_style='success')
button_confirm_activations.on_click(run_next)
display(tab_activations, button_confirm_activations)

In [ ]:
# Create a dict of activations for later on
activations_dict = {}
for i in range(no_of_hidden_layers):
    # For every Hidden Layer: 
    # 1. Read the name of the layer as specified in the Tabs Widget and set it as key
    # 2. Add the chosen activation to the dictionary
    activations_dict[tab_activations.titles[i]] = tab_activations.children[i].value
# Add activation for Output Layer
activations_dict["Output Layer"] = tab_activations.children[no_of_hidden_layers].value

In [ ]:
display(Markdown(string('hd_Output_Layer')))
display(Markdown(string('txt_Output_Layer')))

In [ ]:
display(Markdown(string('hd_Modellübersicht')))
display(Markdown(string('txt_Modellübersicht')))

In [ ]:
# This cell creates a button for showing a model summary 

button_create_model = widgets.Button(description=string('txt_ShowZusammenfassung'), button_style='success', layout = layout)

button_create_model.on_click(run_next_two)
display(button_create_model)

In [ ]:
# All the layers needed are created here

# Input Layer
input_layer = Input(shape=(output_horizon*get_input_ratio(), ))

# First Hidden Layer: input shape = output shape of Input Layer
# units: number of neurons for the Hidden Layer
hidden = Dense(units=no_of_units, activation=activations_dict[tab_activations.titles[0]])(input_layer)

# All other Hidden Layers: Input Shape = output Shape
for i in range(1, no_of_hidden_layers):
    hidden = Dense(units=no_of_units, activation = activations_dict[tab_activations.titles[i]])(hidden)

# Create output layer
outputs = Dense(units=output_horizon, activation=activations_dict["Output Layer"])(hidden)

In [ ]:
# Finalize the model

model = Model(inputs=input_layer, outputs=outputs)

# Define the loss metric that will be minimized by the model during the traning process
loss =  'mae' 

# Define the metrics that are used for evaluation during the training and testing process 
model.compile(loss=loss, optimizer="adam", metrics=['mae', 'mse', 'mape'])

# Save initial model weights
weights_save = model.get_weights()

# Print a model summary
print("\nModel summary:")
print(model.summary())
# Print a list and a graph of the chosen activations
print("\nYou chose the following activations:")
for key in activations_dict.keys():
    print(key, ": ", "\t", activations_dict[key])

chosen_activations = list(activations_dict.values())
plot_activations(chosen_activations, -10, 10)

In [ ]:
display(Markdown(string('hd_Vorhersage')))
display(Markdown(string('txt_Vorhersage')))

In [ ]:
# Slider for specifying the number training/evaluation data split
Input_data_split = widgets.FloatSlider(value=0.8, min=0.05, max=0.95, step=0.05, description=string('lbl_InputSplit'), continuous_update=False, readout_format=".0%", style=style)

# Create Button widget to confirm changes
button_confirm_split = widgets.Button(description=string('lbl_ButtonOK'), button_style='success')

# Display all of the widgets next to each other in a box
display(widgets.HBox(children = [Input_data_split, button_confirm_split]))

button_confirm_split.on_click(run_next)

In [ ]:
data_split = Input_data_split.value

In [ ]:
display(Markdown(string('hd_Training')))
display(Markdown(string('txt_Training')))
display(Markdown(string('txt_Training2')))

In [ ]:
# Input text for specifying the number of epochs during training
Input_no_of_epochs = widgets.BoundedIntText(value=30, min=1, max=500, step=1, description=string('lbl_InputEpochen'), style=style)

# Create Button widget to confirm changes
button_confirm_epochs = widgets.Button(description=string('lbl_ButtonOK'), button_style='success')

# Display all of the widgets next to each other in a box
display(widgets.HBox(children = [Input_no_of_epochs, button_confirm_epochs]))

button_confirm_epochs.on_click(run_next)

In [ ]:
number_of_epochs = Input_no_of_epochs.value
Input_no_of_epochs.observe(run_next, names='value')

In [ ]:
display(Markdown(string('txt_Training3')))

In [ ]:
# This cell creates a button for starting the training

buttonStartTraining = widgets.Button(description=string('txt_StartTraining'),
                            button_style='success')


buttonStartTraining.on_click(run_all_below)
display(buttonStartTraining)

In [ ]:
print("Loading data...")
print("Using {:.0f}% of data as training data.".format(Input_data_split.value*100))
X_train, X_test, Y_train, Y_test, _, y_scaler, date_test_labels = load_data("assets/data/tesa_uebung_ann_data.csv", output_horizon, test_size = 1-Input_data_split.value)

In [ ]:
# Create callback to display training progress
class training_progress_printer(callbacks.Callback):
    def on_epoch_begin(self, epoch, logs =  None):
        
        # For cases with many epochs, only show progress for every nth epoch
        # Determine the n_th epoch depending on number of epochs total
        
        epochs_to_skip = {10 : 1, 100: 5, 500: 20}
        for ele in sorted(epochs_to_skip.keys()):
            if Input_no_of_epochs.value <= ele:
                nth_epoch = epochs_to_skip[ele]
                break
        
        # Always print number of first epoch
        if nth_epoch > 1 and epoch == 0:
            print("Beginning epoch {}".format(epoch+1))
                
        # Print epoch progress (except for last epoch so it doesn't get printed twice) 
        if (epoch+1) % nth_epoch == 0 and (epoch+1) != Input_no_of_epochs.value: 
            print("Beginning epoch {}".format(epoch+1))
        
        # Always print number of last epoch
        if epoch+1 == Input_no_of_epochs.value:
            print("Beginning epoch {}".format(epoch+1))
            print("Training complete")

In [ ]:
print(f"Data loaded. Beginning training with {number_of_epochs} epochs...")
print('Number of training data points:',len(X_train))

# Set model weights to initial weights
model.set_weights(weights_save)

# Train model
history = model.fit(X_train, Y_train, validation_split=0.2, verbose=0, epochs=number_of_epochs, callbacks = [training_progress_printer()])

In [ ]:
# Show results of training using error metrics

# Meanings of metrics defined with model creation
metrics = ["mae", "mse", "rmse"]
meanings = {"mae" : "Mean Absolute Error", "mse" : "Mean Squared Error", "rmse" : "Root Mean Squared Error"}

# tensorflow knows MSE, but not RMSE --> compute RMSE
rmse = np.sqrt(history.history["mse"])
val_rmse = np.sqrt(history.history["val_mse"])

# Add RMSE to metrics dict
metric_data = history.history
metric_data["rmse"] = list(rmse)
metric_data["val_rmse"] = list(val_rmse)

# Create Tab widget and out widgets as children
# Out widgets catch and display output of plotting functions
outs_training = [widgets.Output() for metric in metrics]
tab_training = widgets.Tab(children = outs_training)

for i in range(len(metrics)):
    tab_training.set_title(i, meanings[metrics[i]])
    with outs_training[i]:
        plot_training_history(metric_data, metrics[i])

display(tab_training)

In [ ]:
display(Markdown(string("Abb_Training")))

In [ ]:
display(Markdown(string('txt_Validierung')))
display(Markdown(string('txt_Validierung2')))

In [ ]:
display(Markdown(string('hd_Evaluierung')))
display(Markdown(string('txt_Evaluierung')))

In [ ]:
print("Beginning evaluation...")
Y_predictor = model.predict(X_test)
postprocess_prediction_results(Y_predictor, Y_test, y_scaler, date_test_labels)

In [ ]:
display(Markdown(string("Abb_Evaluation")))